In [1]:
import pandas as pd
import re
pd.set_option('display.max_columns',150)

usaid
dhs - https://apfs-cloud.dhs.gov/forecast/ 
hhs - https://osdbu.hhs.gov/industry/opportunity-forecast
dot - https://www.transportation.gov/new-osdbu/procurement-forecast/export/all?page&_format=csv
gsa - https://hallways.cap.gsa.gov/app/#/dv/federal-business-forecast 
va - https://www.vendorportal.ecms.va.gov/eVP/fco/EntireVA.aspx?Type=Complete

## usaid

In [2]:
# usaid 
usaid_file = 'https://www.usaid.gov/business-forecast/search/download/usaid-business-forecast.xlsx'
usaid_df = pd.read_excel(usaid_file)
usaid_df['Agency'] = 'USAID'
usaid_df['Procurement Tracking Number'] = ''
usaid_df = usaid_df[['Agency','Operating Unit', 'Location','Award Title','Award Description','Code','Total Estimated Cost/Amount Range','Incumbent','Small Business Set-Aside','Anticipated Solicitation Release Date','Category Management Contract Vehicle','Eligibility Criteria','Procurement Tracking Number']]
usaid_df = usaid_df[usaid_df['Location']=='Washington']
usaid_df = usaid_df.rename(columns={"Code": "NAICS", "Anticipated Solicitation Release Date":"Release Date","Operating Unit": "Office", "Small Business Set-Aside": "SB", "Category Management Contract Vehicle":"Vehicle","Eligibility Criteria":"Set Aside","Total Estimated Cost/Amount Range":"Value"})
usaid_df['NAICS'] = usaid_df['NAICS'].astype(str).str[:6]
naics_list = ['541611', '541211', '541219','541511','541990','541618']
usaid_df =  usaid_df[usaid_df['NAICS'].isin(naics_list)]
usaid_df

,Agency,Office,Location,Award Title,Award Description,NAICS,Value,Incumbent,SB,Release Date,Vehicle,Set Aside,Procurement Tracking Number
1,USAID,Bureau for Asia (ASIA),Washington,Asia-Afghanistan Support Services Contract,The contractor shall provide support services ...,541611,$50M - $99.99M,Dexis Consulting,Total set-aside,3/18/2022,OASIS,SB,
33,USAID,Private-Sector Engagement Hub (DDI/PSE),Washington,DDI/PSE_ Private Sector Engagement Modernizati...,DDI/PSE_ New ISC mechanism,541611,$50M - $99.99M,NaN,NaN,2/07/2022,GSA MAS,Any interested organizations within GSA schedule,
43,USAID,Office of the Assistant Administrator (GH/AA),Washington,NextGen Comprehensive Technical Assistance for...,The multiple-award IDIQ contract(s) will be pa...,541611,$2B - $2.49B,NaN,Partial set-aside,10/28/2021,TBD,TBD,
53,USAID,"Office of Management Policy, Budget, and Perfo...",Washington,Management and Agency Priorities,The purpose of Operations Performance for Mana...,541611,$10M - $24.99M,"DT Global, Resonance, Navanti",Total set-aside,4/15/2022,MAS SIN 541611,TBD,


In [3]:
# usda oig
usda_oig_file = 'https://www.dm.usda.gov/smallbus/docs/Office+Of+Inspector+General+FY+2022+Procurement+Forecast.xlsx'
usda_oig_df = pd.read_excel(usda_oig_file)

## dhs

In [4]:
# read file
dhs_file = 'dhs-march22.csv'
dhs_df = pd.read_csv(dhs_file)

# add agency column, re-org & re-name columns
dhs_df['Agency'] = 'DHS'
dhs_df['Award Title'] = ''
dhs_df['Set Aside'] = ''
dhs_df = dhs_df.rename(columns={"Estimated Solicitation Release":"Release Date","Component": "Office", "Small Business Program": "SB", "Contract Vehicle":"Vehicle","Dollar Range":"Value", "Contractor":"Incumbent","Description":"Award Description", "Place of Performance State":"Location","APFS Number":"Procurement Tracking Number"})
dhs_df = dhs_df[['Agency','Office','Location','Award Title','Award Description','NAICS','Value','Incumbent','SB','Release Date','Vehicle','Procurement Tracking Number']]

# clean naics column
dhs_df['NAICS'] = dhs_df['NAICS'].str[:6]

# filter values
state_list = ['DC','MD','VA']
naics_list = ['541611', '541211', '541219','541511','541990','541618']
dhs_df =  dhs_df[dhs_df['Location'].isin(state_list)]
dhs_df =  dhs_df[dhs_df['NAICS'].isin(naics_list)]

# show df
dhs_df

,Agency,Office,Location,Award Title,Award Description,NAICS,Value,Incumbent,SB,Release Date,Vehicle,Procurement Tracking Number
11,DHS,CBP/OIT,VA,,CBP requires an industry partner who is author...,541511,Over $100M,NaN,None,12/03/2021,"GSA Schedule (BPA, DO/TO, PO)",F2020049371
54,DHS,TSA,VA,,TSA Requirements & Capabilities Analysis is in...,541611,$20M to $50M,TSTC,TBD,04/15/2022,"GSA Schedule (BPA, DO/TO, PO)",F2021052978
106,DHS,FEMA,DC,,Federal Emergency Management Agency (FEMA)/Gra...,541611,$10M to $20M,Miracle Systems,SB,02/15/2022,OASIS,F2021054349
109,DHS,TSA,VA,,The scope of work will include:\n \n •\tProgra...,541611,$20M to $50M,NaN,TBD,01/01/2022,OASIS,F2021054420
119,DHS,USCG/CG-912,VA,,The services to be provided by this task order...,541211,$2M to $5M,"Information Technology Coalition, Inc",SDVOSB,03/01/2022,GWAC (GWAC),*F2021054605
...,...,...,...,...,...,...,...,...,...,...,...,...
798,DHS,ICE/M&A/CFO,DC,,The Contractor shall provide the Office of the...,541219,$20M to $50M,Miracle Systems,SB,05/30/2022,"GSA Schedule (BPA, DO/TO, PO)",*F2022057621
803,DHS,USCG/CG-912,DC,,"Program Management, Business and Integrated Lo...",541611,$5M to $10M,Archimedes Global,SDVOSB,05/02/2022,DWAC,F2022057637
804,DHS,USCG/CG-912,DC,,Program Management Support for the United Stat...,541611,$2M to $5M,Archimedes Global,SDVOSB,05/02/2022,DWAC,F2022057638
809,DHS,USCG/CG-912,MD,,The Coast Guard (CG) Surface Acquisitions Logi...,541611,$500K to $1M,F-Concepts,TBD,04/25/2022,Definitive Contract,F2022057659


## hhs

In [5]:
# read file
hhs_file = 'hhs-march22.xlsx'
hhs_df = pd.read_excel(hhs_file)

# add agency column, re-org & re-name columns
hhs_df['Agency'] = 'HHS'
hhs_df['Vehicle'] = ''
hhs_df = hhs_df.rename(columns={'POC Agency':'Office', 'Place of Performance State Code':'Location','Title':'Award Title','Description':'Award Description','Primary NAICS':'NAICS','Total Contract Range':'Value','Incumbent Contractor':'Incumbent','Competition Type':'SB','Target Award Year':'Release Date','Transaction Number':'Procurement Tracking Number'})
hhs_df = hhs_df[['Agency','Office','Location','Award Title','Award Description','NAICS','Value','Incumbent','SB','Release Date','Vehicle','Procurement Tracking Number']]

# clean naics column
hhs_df['NAICS'] = hhs_df['NAICS'].str[:6]

# filter values
state_list = ['DC','MD','VA']
naics_list = ['541611', '541211', '541219','541511','541990','541618']
hhs_df =  hhs_df[hhs_df['Location'].isin(state_list)]
hhs_df =  hhs_df[hhs_df['NAICS'].isin(naics_list)]

# show df
hhs_df

,Agency,Office,Location,Award Title,Award Description,NAICS,Value,Incumbent,SB,Release Date,Vehicle,Procurement Tracking Number
42,HHS,IHS,MD,Silver Level Accreditation,Silver level accreditation and technical assis...,541990,> $25K and < $250K,NaN,TBD,2021.0,,NaN
50,HHS,IHS,MD,Dental Recruitment Program Support,Dental Recruitment Program Support,541990,>= $250K and < $700K,NaN,TBD,2021.0,,NaN
121,HHS,IHS,MD,Health Education Training,Health Education Training,541990,> $25K and < $250K,NaN,TBD,2021.0,,NaN
167,HHS,ORA,MD,Reactor License/Maintenance,Reactor Licenses/Maintenance for cyber-crime i...,541990,> $25K and < $250K,NaN,NaN,2021.0,,ORA-21-C-0306
172,HHS,ORA,MD,Advancing import operations cutting edge techn...,Advancing import operations cutting edge techn...,541511,>= $700K and < $1.5M,NaN,TBD,NaN,,ORA-20-C-0389
...,...,...,...,...,...,...,...,...,...,...,...,...
2732,HHS,IHS,MD,DBH - Objective Review Committee logistics con...,DBH - Objective Review Committee logistics con...,541990,> $25K and < $250K,NaN,TBD,2021.0,,NaN
2783,HHS,IHS,MD,FY21 Security Information and Event Management,FY21 Security Information and Event Management,541990,>= $250K and < $700K,NaN,TBD,2020.0,,NaN
2791,HHS,IHS,MD,DBH - Community Opioid Intervention Pilot Proj...,DBH - Community Opioid Intervention Pilot Proj...,541990,>= $700K and < $1.5M,NaN,TBD,2021.0,,NaN
2807,HHS,IHS,MD,Family Spirit curriculum,"Family Spirit curriculum to include SUD-MAT, c...",541990,> $25K and < $250K,NaN,TBD,2021.0,,NaN


## dot

In [6]:
# read file
dot_file = 'dot-march22.csv'
dot_df = pd.read_csv(dot_file)

# add agency column, re-org & re-name columns
dot_df['Agency'] = 'DOT'
dot_df['Location'] = ''
dot_df['Vehicle'] = ''
dot_df = dot_df.rename(columns={'Procurement Office':'Office', 'Project Title':'Award Title','Description':'Award Description','Estimated Value':'Value','Incumbent/Contract Number':'Incumbent','Competition Type':'SB','RFP Quarter':'Release Date','Sequence Number':'Procurement Tracking Number'})
dot_df = dot_df[['Agency','Office','Location','Award Title','Award Description','NAICS','Value','Incumbent','SB','Release Date','Vehicle','Procurement Tracking Number']]

# clean naics column
dot_df['NAICS'] = dot_df['NAICS'].astype(str).str[:6]

# filter values
naics_list = ['541611', '541211', '541219','541511','541990','541618']
dot_df =  dot_df[dot_df['NAICS'].isin(naics_list)]

# show df
dot_df

,Agency,Office,Location,Award Title,Award Description,NAICS,Value,Incumbent,SB,Release Date,Vehicle,Procurement Tracking Number
2,DOT,OST - Washington DC Office,,NaN,Provide IronPort appliances and maintenance fo...,541511,NaN,NaN,Small Business Set-Aside,1st QTR,,OST 005 CIO
22,DOT,NaN,,NaN,Procurement Support,541611,NaN,NaN,Small Business Set-Aside,1st QTR,,FRA-RAD-007
41,DOT,OST - Washington DC Office,,NaN,Provide Trusted Internet Connection including ...,541511,NaN,NaN,Small Business Set-Aside,1st QTR,,OST 007 CIO
61,DOT,FHWA - Washington DC / Office of Contracts and...,,Artificial Intelligence (AI) for Intelligent T...,This is an overarching Project that covers mul...,541990,$10 million to $20 million,NaN,Full and Open,1st QTR,,2022-080
68,DOT,OST - Washington DC Office,,NaN,"DNS Security deployment including licenses,app...",541511,NaN,NaN,Small Business Set-Aside,1st QTR,,OST 008 CIO
...,...,...,...,...,...,...,...,...,...,...,...,...
2914,DOT,FHWA - Western Federal Lands Highway Division,,NaN,On-site Finance Support Services,541219,"$700,000 to $2 million",NaN,To be determined,4th QTR,,WFL-287
2937,DOT,OST - Washington DC Office,,NaN,Financial statistics analysis of government sp...,541611,"$250,000 to $700,000",NaN,Small Business Set-Aside,TBD,,OST-RTS31-2021-01
2962,DOT,"FMCSA - Washington DC, Office",,NaN,Financial analysis support services.,541611,NaN,NaN,To be determined,TBD,,MC-B-01
2966,DOT,FMCSA - District of Columbia,,Professional Performance Service for Human Res...,"This requirement provides administrative , gra...",541611,"$700,000 to $2 million",NaN,8(a) Competitive,TBD,,MC-2022-004


## gsa

In [19]:
# read file
gsa_file = 'gsa-march22.csv'
gsa_df = pd.read_csv(gsa_file, on_bad_lines='skip')

# add agency column, re-org & re-name columns
gsa_df = gsa_df.rename(columns={'Organization':'Office', 'NAICS Code':'NAICS', 'Place of Performance (State)':'Location', 'Title':'Award Title','Description':'Award Description','Estimated Contract Value (Max.)':'Value','Contractor Name':'Incumbent','Acquisition Strategy/Type of Set-Aside':'SB','Estimated Solicitation Date':'Release Date','Listing ID':'Procurement Tracking Number', 'Procurement Method':'Vehicle'})
gsa_df = gsa_df[['Agency','Office','Location','Award Title','Award Description','NAICS','Value','Incumbent','SB','Release Date','Vehicle','Procurement Tracking Number']]

# clean naics column
gsa_df['NAICS'] = gsa_df['NAICS'].astype(str).str[:6]

# filter values
naics_list = ['541611', '541211', '541219','541511','541990','541618']
gsa_df =  gsa_df[gsa_df['NAICS'].isin(naics_list)]
state_list = ['DC','MD','VA']
gsa_df =  gsa_df[gsa_df['Location'].isin(state_list)]

# show df
gsa_df

,Agency,Office,Location,Award Title,Award Description,NAICS,Value,Incumbent,SB,Release Date,Vehicle,Procurement Tracking Number
1,General Services Administration,FAS-Office of Assisted Acquisition Services,MD,DARRTS Bridge for FY22,DARRTS Bridge for FY22 - Support services for ...,541511,501K to 1M,NaN,None - No Set-Aside Used,2022-01-28,Pre-existing Contract Vehicle - Order - MAS (S...,32557
20,General Services Administration,FAS-Professional Services and Human Capital Ca...,MD,NAVAIR Organizational Systems Analysis & Leade...,NAVAIR Organizational Systems Analysis & Leade...,541611,150K to 250K,NaN,To Be Determined,NaN,NaN,32947
49,General Services Administration,TTS-Technology Transformation Service,DC,USA.gov Bilingual Content Management and Web S...,Bilingual Content Management and Web Support S...,541511,5.1M to 10M,"Cascades Technologies, Inc.",8(a) Competed,NaN,Pre-existing Contract Vehicle - Order - MAS (S...,6394
121,General Services Administration,FAS-Federal Acquisition Service,VA,OCCH Bridge Strategic Consulting Support,The Contractor shall provide all services nece...,541611,2.1M to 5M,NaN,None - No Set-Aside Used,NaN,NaN,31828
135,General Services Administration,FAS-Office of Information Technology Category ...,DC,IT Security Subcategory Management Operations ...,IT Security Subcategory Management Operations ...,541611,1.1M to 2M,FAR 8.405-5,Women-owned Small Business (WOSB),2019-07-15,FAR 8.405-5,26562
...,...,...,...,...,...,...,...,...,...,...,...,...
10518,Department of the Interior,Bureau of Safety and Environmental Enforcement,VA,ABC Program Support,ABC Program Support & Travel,541611,251K to 500K,NaN,To Be Determined,NaN,To Be Determined,22030
10528,Department of the Interior,Bureau of Safety and Environmental Enforcement,VA,Investigate the use of technology that uses vi...,There are numerous technologies that can be us...,541990,251K to 500K,NaN,To Be Determined,NaN,To Be Determined,22057
10817,Department of the Interior,National Park Service,DC,NAMA Custodial Contract,Custodial Contract,541990,1.1M to 2M,NaN,None - No Set-Aside Used; To Be Determined,NaN,NaN,19043
10989,U.S. Office of Personnel Management,CIO/CTO,DC,Digital Service BPA,OCIO requires a vehicle for continued digital ...,541511,Over 100M,NaN,To Be Determined,NaN,Pre-existing Contract Vehicle - Blanket Purcha...,33003


## va

In [18]:
# read file
va_file = 'va-march22.xlsx'
va_df = pd.read_excel(va_file)

# add agency column, re-org & re-name columns
va_df['Agency'] = 'VA' 
va_df['Office'] = ''
va_df['NAICS'] = ''
va_df['Location'] = ''
va_df['Award Title'] = ''
va_df['Incumbent'] = ''
va_df['Procurement Tracking Number'] = ''
va_df = va_df.rename(columns={'Description':'Award Description','Approximate Dollar Range':'Value','Socio-Economic Category':'SB','Forecast Quarter':'Release Date','Procurement Vehicle':'Vehicle'})
va_df = va_df[['Agency','Office','Location','Award Title','Award Description','NAICS','Value','Incumbent','SB','Release Date','Vehicle','Procurement Tracking Number']]


# show df
va_df

,Agency,Office,Location,Award Title,Award Description,NAICS,Value,Incumbent,SB,Release Date,Vehicle,Procurement Tracking Number
0,VA,,,,600-22-2-111-0094 MONTHLY SUBSCRIPTION FOR 1...,,"$25,000.01 to $150,000",,Small Business,Q2,Open Market,
1,VA,,,,36C262-22-AP-0124 Prosthetics-POOL LIFT-644-1...,,"$3,000.01 to $25,000",,Not Set-Aside,Q1,Federal Supply Schedule,
2,VA,,,,36C262-22-AP-1503 / 664-22-2-038-0045,,NaN,,NaN,Q1,NaN,
3,VA,,,,501 | Medical Gas and Tank Rentals,,"$3,000.01 to $25,000",,Not Set-Aside,Q1,Open Market,
4,VA,,,,60 ton BTU Condenser and Air Handler FOR BLDG...,,"$150,000.01 to $5,000,000",,Service Disabled Veteran Owned SB Not Set-Aside,Q1,Open Market Sole Source,
...,...,...,...,...,...,...,...,...,...,...,...,...
19213,VA,,,,Virtual Pharmacy Services \n01/03/2022 thru 01...,,"$3,000.01 to $25,000",,Small Business,Q2,Indefinite Delivery Indefinite Quantity Open M...,
19214,VA,,,,Vyntus CPX/Flex Plan \nService Contract\n,,"$25,000.01 to $150,000",,Not Set-Aside,Q1,Sole Source,
19215,VA,,,,Way Back Inn Option Year 4 FY'22 \n(VA-22-0003...,,"$25,000.01 to $150,000",,Not Set-Aside,Q2,Indefinite Delivery Indefinite Quantity Sole S...,
19216,VA,,,,WLR 556-22-1-2214-0001 Produce Contract (VA-22...,,"$150,000.01 to $5,000,000",,Small Business,Q1,General Service Administration Contract,


# final

In [20]:
# combine dfs
march22forecast = pd.concat([gsa_df,hhs_df,dot_df,dhs_df,va_df], axis=0)
march22forecast = march22forecast.fillna('')
march22forecast

,Agency,Office,Location,Award Title,Award Description,NAICS,Value,Incumbent,SB,Release Date,Vehicle,Procurement Tracking Number
1,General Services Administration,FAS-Office of Assisted Acquisition Services,MD,DARRTS Bridge for FY22,DARRTS Bridge for FY22 - Support services for ...,541511,501K to 1M,,None - No Set-Aside Used,2022-01-28,Pre-existing Contract Vehicle - Order - MAS (S...,32557
20,General Services Administration,FAS-Professional Services and Human Capital Ca...,MD,NAVAIR Organizational Systems Analysis & Leade...,NAVAIR Organizational Systems Analysis & Leade...,541611,150K to 250K,,To Be Determined,,,32947
49,General Services Administration,TTS-Technology Transformation Service,DC,USA.gov Bilingual Content Management and Web S...,Bilingual Content Management and Web Support S...,541511,5.1M to 10M,"Cascades Technologies, Inc.",8(a) Competed,,Pre-existing Contract Vehicle - Order - MAS (S...,6394
121,General Services Administration,FAS-Federal Acquisition Service,VA,OCCH Bridge Strategic Consulting Support,The Contractor shall provide all services nece...,541611,2.1M to 5M,,None - No Set-Aside Used,,,31828
135,General Services Administration,FAS-Office of Information Technology Category ...,DC,IT Security Subcategory Management Operations ...,IT Security Subcategory Management Operations ...,541611,1.1M to 2M,FAR 8.405-5,Women-owned Small Business (WOSB),2019-07-15,FAR 8.405-5,26562
...,...,...,...,...,...,...,...,...,...,...,...,...
19213,VA,,,,Virtual Pharmacy Services \n01/03/2022 thru 01...,,"$3,000.01 to $25,000",,Small Business,Q2,Indefinite Delivery Indefinite Quantity Open M...,
19214,VA,,,,Vyntus CPX/Flex Plan \nService Contract\n,,"$25,000.01 to $150,000",,Not Set-Aside,Q1,Sole Source,
19215,VA,,,,Way Back Inn Option Year 4 FY'22 \n(VA-22-0003...,,"$25,000.01 to $150,000",,Not Set-Aside,Q2,Indefinite Delivery Indefinite Quantity Sole S...,
19216,VA,,,,WLR 556-22-1-2214-0001 Produce Contract (VA-22...,,"$150,000.01 to $5,000,000",,Small Business,Q1,General Service Administration Contract,


In [21]:
filtered_forecast = march22forecast[(march22forecast['Award Title'].str.contains("financ|budget|polic|data|account|grant|risk|internal control|erm|enterprise risk management|a-123|a123|audit|improper payment|fraud|compliance|performance|ipera|iperia|business process|cfo|automation|strategic management|change management", flags=re.IGNORECASE)) | (march22forecast['Award Description'].str.contains("financ|budget|polic|data|account|grant|risk|internal control|erm|enterprise risk management|a-123|a123|audit|improper payment|fraud|compliance|performance", flags=re.IGNORECASE))]
filtered_forecast

,Agency,Office,Location,Award Title,Award Description,NAICS,Value,Incumbent,SB,Release Date,Vehicle,Procurement Tracking Number
264,Department of Labor,OASAM,DC,Financial Statement Preparation and Accounting...,Financial Statement Preparation and Accounting...,541611,10.1M to 20M,"Verdi Consulting, Inc.",None - No Set-Aside Used,,Pre-existing Contract Vehicle - Blanket Purcha...,29908
270,General Services Administration,CIO-Office of the Chief Information Officer,DC,Financial Management IT Tech Support,This task order relates to the development and...,541611,20.1M to 50M,,To Be Determined,,,32602
285,Department of the Interior,Bureau of Safety and Environmental Enforcement,VA,Artificial Intelligence Advancement/Data Scien...,Develop the AI concepts created by the NASA Ad...,541511,2.1M to 5M,,None - No Set-Aside Used,,To Be Determined,29177
801,General Services Administration,"FAS-Office of Travel, Transportation, and Logi...",DC,Transportation Audits Management System Suppor...,Process and Performance Improvement for Transp...,541611,251K to 500K,Novaces,Small Business,2021-09-15,Pre-existing Contract Vehicle - Order-MAS Sche...,31642
861,General Services Administration,CIO-Office of the Chief Information Officer,DC,PBS Data Integration and Repository Management...,O&M Support,541511,501K to 1M,,To Be Determined,,,31518
...,...,...,...,...,...,...,...,...,...,...,...,...
18612,VA,,,,UDO: 36C26018A0052/36C26020N0552/116S06147 FMS...,,,,Small Business,Q1,Blanket Purchase Agreement,
18779,VA,,,,Modification of Johnson Police Security Upgrad...,,"$25,000.01 to $150,000",,,Q2,,
18783,VA,,,,Telehealth IPad Cellular Data Plan (VA-22-0004...,,"$25,000.01 to $150,000",,,Q2,,
18919,VA,,,,695-22-2-944-0338 COVID ThermoFisher Reagents ...,,"$150,000.01 to $5,000,000",,Not Set-Aside,Q2,Indefinite Delivery Indefinite Quantity Sole S...,


In [22]:
filtered_forecast.to_csv('sample_filtered_procurement_forecast.csv')